<center> </center>

<center><font size=5 face="Helvetica" color=#306998><b>
Fourier Modal Method
</b></font></center>

<center><font face="Helvetica" size=2><b>Ang Chen</b></font></center>
<center><font face="Helvetica" size=3>May, 2024</font></center>

***

# Introduction

The basis idea of Fourier Modal Method (FMM) is quite simple: 
 * The electromagnetic fields are first solved as eigenfunctions of Maxwell's equations in the interior of the grating region where the periodic permittivity varies.
 * Then the total fields are expressed as superpositions of the eigenfunctions.
 * The final solution of the grating problem is obtained by matching the electromagnetic boundary conditions at the boundaries of the grating region.
 
The most importtant step in FMM is the solution of the eigenfunctions.
In physical and engineering terms, the eigenfunctions of a mechanical or electromagnetic system are called modes,  hence the name *modal method*.

In order to seek the eigenfunctions, there is an approach expanding both fields and permittibity functions into Fourier series, which transforms the electromagnetic boundary value problem in real space into a matrix eigenvalue problem in denumerably infinite Fourier space. 
Among many names, the name *Fourier Modal Method* is the most appropriate one due to its accurate description of the method's characteristics.

Our goal here is to develop the fastest and most accurate FMM algorithm for scientific resarch and industrial applications such as optical critical dimension metrology in semiconductor manufacturing, with the help of multi-core CPU and GPU computing.
To achieve this, we will use the libraries and platforms such as MPI, CUDA, and OpenMP, which are widely used in scientific computing and provide efficient parallel computing capabilities.

Almost all the algorithms in computational photonics (or more generally, computational electromagnetics) starts with the 
Maxwell's equations, with the SI units as follows:
\begin{align}       
\nabla\cdot\textbf{D}&=\rho, \tag{1.1a} \\
\nabla\times\textbf{E} &= -\frac{\partial\textbf{B}}{\partial t}, \tag{1.1b} \\
\nabla\times\textbf{H} &= \textbf{J} + \frac{\partial\textbf{D}}{\partial t}, \tag{1.1c} \\
\nabla\cdot\textbf{B} &= 0. \tag{1.1d} 
\end{align}
And coresponding material relations are:
\begin{align}
\textbf{D} &= \epsilon_0\epsilon\textbf{E}, \tag{1.2a} \\
\textbf{B} &= \mu_0\mu\textbf{H}. \tag{1.2b}      
\end{align}

For full description of FMM, we can divide the situations into two categories: structures with and without
sources.
Ref. [1] describes the cases without sources, and Ref. [2] does with sources.

This note is organized as follows (I will continuously update the following sections): 
 * Section 2 describes the mathematical framework of FMM following $\texttt{S4}$ paper [1], together with the previous works in the field. 
 * Section 3 introduces the $\texttt{fmmax}$ paper [2], putting foward the FMM applications in simulating and optimizing the field of micro-LEDs, and extra simulated capabilities of anisotropic materials.
 * Section 4 builds both python and C++ codes for FMM for different applications in scientific computing and industrial applications. 

# Formulation of S4

## Geometric definitions

Througout this note, the coordinate system is oriented such that the $z$-axis is normal to the layers of the
structure, and the structure is assumed to be periodic in the $xy$-plane with primitive lattice vectors
$\textbf{l}_1$ and $\textbf{l}_2$, as shown in the cover.
The transverse coordinates in the $xy$-plane are lumped into a vector $\textbf{r}$. 
Layer $i$ has the thickness $d_i$ and extends from $z = z_i$ to $z = z_i + d_i$, with layer 1 extending from 
$z = z_0 = 0$ to $z = d_1$ as an example. 
The infinite half-space $z < 0$ in front of the structure is denoted layer 0, and the infinite half-space behind the
structure is denoted layer $L$.
Each layer is assumed invariant in the z-direction; 
that is, in layer $i$, the spatial dielectric distribution $\epsilon(x, y, z$) is constant for fixed $x$ and $y$, and
$z_i < z < z_i + d_i$.

<img src="../imgs/real_reciprocal_spaces.png" alt="Real and Reciprocal Spaces"/>

For the reciprocal lattice of the Fourier domain, we first denote a real space primitive lattice vector matrix
whose columns are $\textbf{l}_1$ and $\textbf{l}_2$, as shown in the upper figure:
\begin{equation}
\tag{2.1}
L_r = 
\begin{bmatrix}
\textbf{l}_1 & \textbf{l}_2
\end{bmatrix} =
\begin{bmatrix}
l_{1x} & l_{2x} \\
l_{1y} & l_{2y}
\end{bmatrix}.
\end{equation}
We can also define the matrix of the reciprocal lattice vectors as
\begin{equation}
\tag{2.2}
L_k = 
\begin{bmatrix}
\textbf{n}_1 & \textbf{n}_2
\end{bmatrix} =
\begin{bmatrix}
n_{1x} & n_{2x} \\
n_{1y} & n_{2y}
\end{bmatrix}.
\end{equation}
The primitive reciprocal lattice vectors can be constructed with the following recipe:
\begin{equation}
\tag{2.3}
\textbf{n}_1 = \frac{2\pi \textbf{l}_2\times\textbf{l}_3}{\textbf{l}_1\cdot\left(\textbf{l}_2\times\textbf{l}_3\right)},\quad
\textbf{n}_2 = \frac{2\pi \textbf{l}_3\times\textbf{l}_1}{\textbf{l}_1\cdot\left(\textbf{l}_2\times\textbf{l}_3\right)}.
\end{equation}
Here the third primitive lattice vector is $\textbf{l}_3 = \hat{z}$, and hence we can reach
\begin{equation}
\tag{2.4}
L_k = 
\frac{2\pi}{\textbf{l}_1\cdot\left(\textbf{l}_2\times\hat{z}\right)}
\begin{bmatrix}
\textbf{l}_2\times\hat{z} & \hat{z}\times\textbf{l}_1
\end{bmatrix} =
\frac{2\pi}{l_{1x}l_{2y}-l_{2x}l_{1y}}
\begin{bmatrix}
l_{2y} & -l_{1y} \\
-l_{2x} & l_{1x}
\end{bmatrix} = 
2\pi \left(L_r^{-1}\right)^\text{T}.
\end{equation}

## Maxwell's equations in a layer

To calculate reflection, transmission, or absorption spectra of structures, incident radiation
from layer $0$ is assumed to be a plane wave propagating in positive $z$ direction
towards layer $1$, whose incident wavevector is $\textbf{k}_\text{inc}$ with in-plane component $\textbf{k}$.

### Units and conventions

Assuming a harmonic time dependence $e^{-i\omega t}$, Maxwell's equations in a *dielectric* medium 
together with the material relations can be written as (here for most dielectrics, $\mu=1$ is a good assumption):
\begin{align}
\nabla\cdot\epsilon_0\epsilon\textbf{E} &= 0, \tag{2.5a} \\
\nabla\cdot\textbf{H} &= 0, \tag{2.5b} \\
\nabla\times\textbf{E} &= i\omega\mu_0\textbf{H}, \tag{2.5c}  \\
\nabla\times\textbf{H} &= -i\omega\epsilon_0\epsilon\textbf{E}. \tag{2.5d} 
\end{align}
Note that Eq. (2.5a) and Eq. (2.5b) are automatically
implied by Eq. (2.5d)and Eq. (2.5c), respectively.
In order to make $\epsilon_0, \mu_0$ and $c$ drop out, we need to scale $\sqrt{\mu_0\epsilon_0}\omega_\text{SI}\rightarrow\omega$,  
$\sqrt{\mu_0\epsilon_0}\textbf{H}_\text{SI}\rightarrow\textbf{H}$ and $\textbf{E}_\text{SI}\rightarrow\textbf{E}$. 
This change of units brings the electric and magnetic fields onto the same scale, and the temporal and spatial frequency scales onto the
same scale, also providing better numerical conditioning in the implementation.
With these new units, Eqs. (2.5c) and (2.5d) can be written as
\begin{align}
\nabla\times\textbf{E} &= i\omega\textbf{H}, \tag{2.6a}  \\
\nabla\times\textbf{H} &= -i\omega\epsilon\textbf{E}. \tag{2.6b} 
\end{align}

### Fourier transforms

Because of the periodicity in $xy$-plane and separability of the $z$-axis, the fields must have the form [4] (detailed derivation can be found in Appendix A):
\begin{equation}
\tag{2.7}
\textbf{H}_\textbf{k}(\textbf{r},z) = 
\sum_\textbf{G} \textbf{H}_\textbf{G}(z) e^{i\left(\textbf{k}+\textbf{G}\right)\cdot\textbf{r}}.
\end{equation}
An analogous equation holds for $\textbf{E}$.
A real simulation needs the trunction to a finite set of $\textbf{G}$, which is in principle the only ``discretization'' for the FMM.

# A Field form derivation

In the section ``Discrete Translational Symmetry'' of Ref. [4], the authors
have derived the field form of a nanostructure that is repetitive in one direction ($y$-axis).
Here following the same procedure, we will derive the field form for a structure that is periodic  in $xy$-plane.
The eigenfunctions in $xy$-plane are plane waves:
\begin{equation}
\tag{A.1}
\hat{T}_{\textbf{R}_{m,n}} e^{i\textbf{k}\cdot\textbf{r}} = 
e^{i\textbf{k}\cdot(\textbf{r}-\textbf{R}_{m,n})} =
e^{-i\textbf{k}\cdot\textbf{R}_{m,n}} e^{i\textbf{k}\cdot\textbf{r}},
\end{equation}
where $\textbf{R}_{m,n} = m\textbf{l}_1 + n\textbf{l}_2$.
Given $\textbf{k}$, $\textbf{k}+\textbf{G}_{p,q}$ also gives operator $\hat{T}_{\textbf{R}_{m,n}}$ the same eigenvalue 
$e^{-i\textbf{k}\cdot\textbf{R}_{m,n}}$ with
\begin{equation}
\tag{A.2}
\textbf{G}_{p,q} = 
p\textbf{n}_1 + q\textbf{n}_2 =
\begin{bmatrix}
\textbf{n}_1 & \textbf{n}_2
\end{bmatrix} 
\begin{bmatrix}
p \\ q
\end{bmatrix} = 
L_k \mathbb{Z}^2.
\end{equation}
All of the modes with wave vectors of the form $\textbf{k}+\textbf{G}_{p,q}$, form a degenerate set whose eigenvalues are
all equal to $e^{-i\textbf{k}\cdot\textbf{R}_{m,n}}$.
Since any linear combination of these degenerate eigenfunctions is itself an eigenfunction with the same eigenvalue, 
we can take linear combinations of our original modes to put them in the form
\begin{equation}
\tag{A.3}
\textbf{H}_\textbf{k}(\textbf{r},z) = 
\sum_{p,q} c_{p,q}(z) e^{i\left(\textbf{k}+\textbf{G}_{p,q}\right)\cdot\textbf{r}} =
\sum_\textbf{G} \textbf{H}_\textbf{G}(z) e^{i\left(\textbf{k}+\textbf{G}\right)\cdot\textbf{r}}.
\end{equation}
Here we denote the expansion coefficients $c_{p,q}(z)$ with $\textbf{H}_\textbf{G}(z)$, and summation with $\textbf{G}$.


# References

1. Victor Liu and Shanhui Fan. *S4: A free electromagnetic solver for layered periodic structures*. Computer Physics Communications **183**: 2233, 2012.

2. D. M. Whittaker and I. S. Culshaw. *Scattering-matrix treatment of patterned multilayer photonic structures*. Physical Review B **60**: 2610, 1999.

3. Martin F. Schubert and Alec M. Hammond. *Fourier modal method for inverse design of metasurface-enhanced micro-LEDs*. Optics Express **31**: 42945, 2023.

4. J. D. Joannopoulos, S. G. Johnson, J. N. Winn and R. D. Meade. *Photonic Crystals: Molding the Flow of Light* 2nd ed (Princeton University Press, 2008).

***